# demo3DWF
* Step 1: Train 3D generative model of the nucleus, cell shape, and lysosome using 30 LAMP2 images in the Murphy Lab 3D HeLa dataset.
* Step 2: Synthesize one 3D image with nuclear, cell shape, and nucleolar channels from nucleolar model with sampling method set to render nucleoli as ellipsoids without convolution. The model was trained from the Murphy Lab 3D HeLa dataset.

## Input 
* a valid CellOrganizer model file

## Output
* three TIFF files (nuclear, cell shape, and nucleolar channels)

In [4]:
from cellorganizer.tools import slml2img
from cellorganizer.tools import img2slml
import glob
import urllib.request
import tarfile 
import os


#### PERFORMING IMAGES -> SLML MODEL ######################
if not os.path.exists('../../../images/HeLa/3D/processed'):
    os.system('wget -nc --quiet http://www.cellorganizer.org/Downloads/v2.8.0/docker/images/demo3D12.tgz;mkdir -p ../../../images/HeLa/3D/processed && tar -xvf demo3D12.tgz -C ../../../images/HeLa/3D/processed/ && rm -f demo3D12.tgz')

pattern = 'LAMP2'
dimensionality = '3D'

options = {}
options['model.name'] = 'all'
options['model.id'] = 'num2str(now)'

options['nucleus.type'] = 'cylindrical_surface'
options['nucleus.class'] = 'nuclear_membrane'
options['nucleus.id'] = 'num2str(now)'

options['cell.type'] = 'ratio'
options['cell.id'] = 'num2str(now)'
options['cell.class'] = 'cell_membrane'

options['protein.type'] = 'gmm'
options['protein.class'] = 'vesicle'
options['protein.id'] = 'num2str(now)'

if options['protein.class'] == 'vesicle' :
    options['protein.cytonuclearflag'] = 'nuc'
else:
    options['protein.cytonuclearflag'] = 'cyto'

options['verbose'] = True
options['debug'] = False
options['documentation.description'] = 'This model has been trained using demo3D12 from CellOrganizer'

directory = '../../../images/HeLa/3D/processed/'
dnaImagesDirectoryPath,cellImagesDirectoryPath,proteinImagesDirectoryPath,options['masks']=[],[],[],[]
for i in range(1,11):
    dnaImagesDirectoryPath.append(directory+"LAM_cell"+str(i)+"_ch0_t1.tif" )
    cellImagesDirectoryPath.append(directory+"LAM_cell"+str(i)+"_ch1_t1.tif")
    proteinImagesDirectoryPath.append(directory+"LAM_cell"+str(i)+"_ch2_t1.tif")
    options['masks'].append(directory+"LAM_cell"+str(i)+"_mask_t1.tif")

    
options['train.flag'] = 'all'
options['model.resolution'] = [0.049, 0.049, 0.2000]
options['model.filename'] = 'LAMP2.xml'

img2slml(dimensionality, dnaImagesDirectoryPath, cellImagesDirectoryPath, proteinImagesDirectoryPath, options)


options = {}
options['targetDirectory'] = 'pwd'
options['prefix'] = 'img'
options['compression'] = 'lzw'
options['sampling.method']= 'disc'
options['debug'] = False
options['verbose'] = True
options['display'] = False
filenames = ['lamp2.mat']

#### PERFORMING IMAGES -> SLML MODEL ######################
slml2img( filenames, options )